In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import *

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

### Обработка данных

In [2]:
df = pd.read_csv('2liver.csv')
df

,Age,Gender,TotalBilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset123
0,65,Female,0.7,0.1,187.0,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699.0,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490.0,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182.0,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195.0,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
582,38,Male,1.0,0.3,216.0,21,24,7.3,4.4,1.50,2
583,38,Male,NaN,0.3,216.0,21,24,7.3,4.4,1.50,2
584,38,Male,1.0,0.3,216.0,21,24,7.3,4.4,1.50,2
585,40,Male,0.6,0.1,98.0,35,31,6.0,3.2,1.10,1


Информация о датасете 2liver.csv 

Содержит информацию о заболеваемости печени:
- Возраст пациента
- Пол пациента
- Общий билирубин
- Прямой билирубин
- Щелочная фосфатаза
- Аламиноаминотрансфераза
- Аспартатаминотрансфераза
- Всего белков
- Альбумин
- Соотношение альбуминов и глобулинов
- Цель: поле, используемое для разделения данных на два набора (пациент с заболеванием печени
или без заболевания).


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         587 non-null    int64  
 1   Gender                      587 non-null    object 
 2   TotalBilirubin              585 non-null    float64
 3   Direct_Bilirubin            587 non-null    float64
 4   Alkaline_Phosphotase        586 non-null    float64
 5   Alamine_Aminotransferase    587 non-null    int64  
 6   Aspartate_Aminotransferase  587 non-null    object 
 7   Total_Protiens              587 non-null    float64
 8   Albumin                     587 non-null    float64
 9   Albumin_and_Globulin_Ratio  583 non-null    float64
 10  Dataset123                  587 non-null    object 
dtypes: float64(6), int64(2), object(3)
memory usage: 50.6+ KB


In [4]:
for i in df.columns:
    print(i, sorted(df[i].unique()), end = '\n\n', sep = '\n')

Age
[4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 78, 84, 85, 90]

Gender
['Female', 'Mal', 'Male']

TotalBilirubin
[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.4, 4.5, 4.7, 4.9, 5.0, 5.2, 5.3, 5.5, 5.7, 5.8, 5.9, 6.2, 6.3, 6.6, 6.7, 6.8, 7.1, 7.3, 7.4, 7.5, 7.7, 7.9, 8.0, 8.2, 8.6, 8.7, 8.9, 9.4, 10.2, 10.6, 10.9, 11.0, 11.1, 11.3, 11.5, 12.1, 12.7, 14.1, 14.2, 14.5, 14.8, 15.0, 15.2, 15.6, 15.8, 15.9, 16.4, 16.6, 16.7, 17.3, 17.7, 18.0, 18.4, 18.5, 19.6, 19.8, 20.0, 20.2, 22.5, 22.6, 22.7, 22.8, 23.0, 23.2, 23.3, 25.0, 26.3, 27.2, 27.7, 30.5, 30.8, 32.6, 42.8, 75.0, nan]

Direct_Bilirubin
[0.1, 0.2, 0.3, 0.4, 0.5, 

Произведена замена названий колонок, проведён Label encoding (0 - Женщина, 1 - Мужчина)

In [5]:
df = df.rename(columns={'Dataset123': 'Bool'})
df = df.dropna()
df.loc[:, 'Aspartate_Aminotransferase'] = df['Aspartate_Aminotransferase'].replace('3a4','34')
df.loc[:, 'Bool'] = df['Bool'].replace('yes','1')
df.loc[:, 'Gender'] = df['Gender'].replace('Female','0').replace('Male','1').replace('Mal','1')
df = df.apply(pd.to_numeric, errors='coerce')
df = df.astype('float')

In [6]:
X = df.drop(columns=['Bool']) 
y = df['Bool']

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled,y,test_size=0.25,
                                                    random_state=42)

Были разработаны следующие модели:
- метод k- ближайших соседей
- логистическая регрессия
- дерево решений
- случайный лес  


In [9]:
KNeighborsClassifier = KNeighborsClassifier()
KNeighborsClassifier.fit(X_train, Y_train)

y_pred_neighbors = KNeighborsClassifier.predict(X_test)

acc = accuracy_score(Y_test, y_pred_neighbors)
precision = precision_score(Y_test, y_pred_neighbors)
recall = recall_score(Y_test, y_pred_neighbors)
f1 = f1_score(Y_test, y_pred_neighbors)
balanced_accuracy = balanced_accuracy_score(Y_test, y_pred_neighbors)
print(classification_report(Y_test, y_pred_neighbors))
print('balanced accuracy: ', balanced_accuracy)
print('accuracy: ', acc)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

              precision    recall  f1-score   support

         1.0       0.77      0.81      0.79       105
         2.0       0.43      0.38      0.40        40

    accuracy                           0.69       145
   macro avg       0.60      0.59      0.60       145
weighted avg       0.68      0.69      0.68       145

balanced accuracy:  0.5922619047619048
accuracy:  0.6896551724137931
precision:  0.7727272727272727
recall:  0.8095238095238095
f1:  0.7906976744186046


In [13]:
confusion_matrix(Y_test,y_pred_neighbors)

array([[85, 20],
       [25, 15]], dtype=int64)

In [10]:
LogisticRegression = LogisticRegression()
LogisticRegression.fit(X_train, Y_train)
y_pred_log_regress = LogisticRegression.predict(X_test)

acc = accuracy_score(Y_test, y_pred_log_regress)
precision = precision_score(Y_test, y_pred_log_regress)
recall = recall_score(Y_test, y_pred_log_regress)
f1 = f1_score(Y_test, y_pred_log_regress)
balanced_accuracy = balanced_accuracy_score(Y_test, y_pred_log_regress)
print(classification_report(Y_test, y_pred_log_regress))
print('balanced accuracy: ', balanced_accuracy)
print('accuracy: ', acc)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

              precision    recall  f1-score   support

         1.0       0.74      0.94      0.83       105
         2.0       0.45      0.12      0.20        40

    accuracy                           0.72       145
   macro avg       0.60      0.53      0.51       145
weighted avg       0.66      0.72      0.65       145

balanced accuracy:  0.5339285714285714
accuracy:  0.7172413793103448
precision:  0.7388059701492538
recall:  0.9428571428571428
f1:  0.8284518828451883


In [14]:
confusion_matrix(Y_test,y_pred_log_regress)

array([[99,  6],
       [35,  5]], dtype=int64)

In [11]:
DecisionTreeClassifier = DecisionTreeClassifier()
DecisionTreeClassifier.fit(X_train, Y_train)
y_pred_tree = DecisionTreeClassifier.predict(X_test)

acc = accuracy_score(Y_test, y_pred_tree)
precision = precision_score(Y_test, y_pred_tree)
recall = recall_score(Y_test, y_pred_tree)
f1 = f1_score(Y_test, y_pred_tree)
balanced_accuracy = balanced_accuracy_score(Y_test, y_pred_tree)
print(classification_report(Y_test, y_pred_tree))
print('balanced accuracy: ', balanced_accuracy)
print('accuracy: ', acc)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

              precision    recall  f1-score   support

         1.0       0.74      0.94      0.83       105
         2.0       0.45      0.12      0.20        40

    accuracy                           0.72       145
   macro avg       0.60      0.53      0.51       145
weighted avg       0.66      0.72      0.65       145

balanced accuracy:  0.6386904761904761
accuracy:  0.6896551724137931
precision:  0.8061224489795918
recall:  0.7523809523809524
f1:  0.7783251231527094


In [15]:
confusion_matrix(Y_test,y_pred_tree)

array([[79, 26],
       [19, 21]], dtype=int64)

In [12]:
RandomForestClassifier = RandomForestClassifier()
RandomForestClassifier.fit(X_train, Y_train)
y_pred_forest = RandomForestClassifier.predict(X_test)

acc = accuracy_score(Y_test, y_pred_forest)
precision = precision_score(Y_test, y_pred_forest)
recall = recall_score(Y_test, y_pred_forest)
f1 = f1_score(Y_test, y_pred_forest)
balanced_accuracy = balanced_accuracy_score(Y_test, y_pred_forest)
print(classification_report(Y_test, y_pred_forest))
print('balanced accuracy: ', balanced_accuracy)
print('accuracy: ', acc)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

              precision    recall  f1-score   support

         1.0       0.76      0.86      0.80       105
         2.0       0.42      0.28      0.33        40

    accuracy                           0.70       145
   macro avg       0.59      0.57      0.57       145
weighted avg       0.66      0.70      0.67       145

balanced accuracy:  0.5660714285714286
accuracy:  0.696551724137931
precision:  0.7563025210084033
recall:  0.8571428571428571
f1:  0.8035714285714286


In [20]:
sns.heatmap(confusion_matrix(Y_test,y_pred_forest), annot=True, cmap='cold')
plt.title("Матрица неточностей (Confusion Matrix)", fontsize=14);
plt.ylabel('Истинные значения')
plt.xlabel('Предсказанные значения')

KeyError: "'cold' is not a known colormap name"


- True Positive – TP («истинно положительные») [0,0] – прогноз модели 1, реальное значение 1;
- False Positive – FP («ложно положительные») [0,1] – прогноз модели 1, реальное значение 0;
- False Negative – FN («ложно отрицательные») [1,0] – прогноз модели 0, реальное значение 1;
- True Negative– TP («истинно отрицательные») [1,1]– прогноз модели 0, реальное значение 0.